## `ws` (water state) Variable Summaries

This notebook will orchestrate the monthly decadal mean summaries of the following variables from the `ws` set of files:

 - SWE (Snow water equivalent, mm)
 - IWE (Ice water equivalent, mm)
 - SM1 (Soil moisture layer1, mm)
 - SM2 (Soil moisture layer1, mm)
 - SM3 (Soil moisture layer3, mm)

All this notebook does is stage the files, set up a Dask client, and then iterate through all models, scenarios, decades, variables, and months to produce the summary GeoTIFFs.

In [8]:
import os
import xarray as xr
import rasterio as rio
import numpy as np
import dask
import dask.array as da
from dask.distributed import Client
from dask import delayed
from pyproj import Proj, Transformer, CRS
from tqdm.auto import tqdm
from pathlib import Path
from wrf import PolarStereographic

# local
from config import DATA_DIR, OUTPUT_DIR
from config import models, scenarios, variable_di, precision_di, months, unit_di, summary_di, mo_names
from compute_summaries import *

In [9]:
var_set = "ws"

In [10]:
paths = list(DATA_DIR.glob(f"*{var_set}*.nc"))
assert len(paths) == 3000 # value learned from EDA notebook

In [11]:
process_group_di = {}
for model in models:
    process_group_di[model] = {}
    for scenario in scenarios:
        process_group_di[model][scenario] = []

for fp in paths:
    fp_model = fp.name.split("_")[0]
    fp_scenario = fp.name.split("_")[1]
    process_group_di[fp_model][fp_scenario].append(fp)

for model in models:
    for scenario in scenarios:
        assert len(process_group_di[model][scenario]) == 150 # value learned from EDA notebook

In [12]:
# create dask client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 251.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35393,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 251.72 GiB
Comm: tcp://127.0.0.1:32992,Total threads: 4
Dashboard: http://127.0.0.1:43527/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:42716,


In [13]:
%%time

for model in models:
    for scenario in scenarios:

        ncube = mfload_all_netcdf_data(process_group_di[model][scenario])
        projcube, wrf_raster_profile = project_datacube(ncube)

        # keeping this snippet for convenient testing - just process a single decade
        # for decade_start in tqdm(range(1950, 1960, 10), desc=f"Processing {model} {scenario} data..."):
        for decade_start in tqdm(range(1950, 2100, 10), desc=f"Processing {model} {scenario} data..."):
            decade_slice = slice_by_decade(projcube, decade_start)
            
            for climvar in tqdm(variable_di[var_set].keys(), desc=f"Processing {decade_start}s"):
                decadal_means_of_monthly_summaries = compute_monthly_summaries(decade_slice, var_set, climvar)
                
                for month in months:
                    month_array = array_from_monthly_summary(decadal_means_of_monthly_summaries, climvar, month)
                    output_filename = make_output_filename(climvar, model, scenario, month, decade_start)
                    write_raster_to_disk(output_filename, wrf_raster_profile, month_array)

        projcube.close()

Processing ACCESS1-3 rcp45 data...:   0%|          | 0/15 [00:00<?, ?it/s]

Processing 1950s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1960s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1970s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1980s:   0%|          | 0/5 [00:00<?, ?it/s]

...

Processing 2060s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 2070s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 2080s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 2090s:   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 44min 1s, sys: 3min 49s, total: 47min 50s
Wall time: 1h 3min 59s


In [14]:
client.close()